In [7]:
#Import the required libraries

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())


In [9]:
#Read the dataset and rename the columns as per the names given in the file

sqladb=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',names=['age','work-class','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','>50K, <=50K'])

#Define the function to trim all spaces from dataframe column values
def trimAllColumns(df):
    trimStrings = lambda x : x.strip() if type(x) is str else x
    return df.applymap(trimStrings)

#Call the trim function on dataframe to trim all spaces from column values
sqladb = trimAllColumns(sqladb)

#Replace '-' with '_' in column names

sqladb.columns = sqladb.columns.str.replace('-', '_')

In [10]:
#Question 1: Select 10 records from the adult sqladb

pysqldf("SELECT * FROM sqladb LIMIT 10;")

,age,work_class,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,">50K, <=50K"
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [14]:
#Question 2:  Show the average hours per week of all men who are working in private sector

#Calculate the average after grouping by sex = 'Male' and work_class = 'Private'
pysqldf("SELECT avg(hours_per_week) FROM sqladb GROUP BY sex, work_class having sex = 'Male' and work_class='Private'")

,avg(hours_per_week)
0,42.221226


In [17]:
#Question 3: Show the frequency table for education, occupation and relationship, separately

#Calculate the frequency table for education by grouping on education and doing a count of values
print("The frequency table for education is:\n")
print(pysqldf("SELECT education, count(*) as count FROM sqladb group by education"))

#Calculate the frequency table for occupation by grouping on occupation and doing a count of values
print("The frequency table for occupation is:\n")
print(pysqldf("SELECT occupation, count(*) as count FROM sqladb group by occupation"))

#Calculate the frequency table for relationship by grouping on relationship and doing a count of values
print("The frequency table for relationship is:\n")
print(pysqldf("SELECT relationship, count(*) as count FROM sqladb group by relationship"))

The frequency table for education is:

       education  count
0           10th    933
1           11th   1175
2           12th    433
3        1st-4th    168
4        5th-6th    333
5        7th-8th    646
6            9th    514
7     Assoc-acdm   1067
8      Assoc-voc   1382
9      Bachelors   5355
10     Doctorate    413
11       HS-grad  10501
12       Masters   1723
13     Preschool     51
14   Prof-school    576
15  Some-college   7291
The frequency table for occupation is:

           occupation  count
0                   ?   1843
1        Adm-clerical   3770
2        Armed-Forces      9
3        Craft-repair   4099
4     Exec-managerial   4066
5     Farming-fishing    994
6   Handlers-cleaners   1370
7   Machine-op-inspct   2002
8       Other-service   3295
9     Priv-house-serv    149
10     Prof-specialty   4140
11    Protective-serv    649
12              Sales   3650
13       Tech-support    928
14   Transport-moving   1597
The frequency table for relationship is:

     re

In [20]:
#Question 4: People who are married, working in private sector and having a masters degree

#Find the count of people by filtering data on marital_status, work_class and eduction
print("Count of people who are married, working in private sector and having a masters degree:")
print(pysqldf("SELECT count(*) as count FROM sqladb WHERE marital_status = 'Married' and work_class = 'Private' and education = 'Masters'"))

Count of people who are married, working in private sector and having a masters degree:
   count
0      0


In [22]:
#Question 5: The average, minimum and maximum age group for people working in different sectors

#Find the average for age, and minimum/maximum value for age by grouping on work_class
pysqldf("SELECT work_class as 'Sector', avg(age) as 'Average Age', min(age) as 'Min Age', max(age) as 'Max Age' FROM sqladb GROUP BY work_class")

,Sector,Average Age,Min Age,Max Age
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436055,17,81
8,Without-pay,47.785714,19,72


In [42]:
#Question 6: Calculate age distribution by country

#Group by native_country and age to get the age distribution by country
print("Age distribution by Country:")
print(pysqldf("SELECT native_country as country, age FROM sqladb GROUP BY native_country, age order by native_country desc"))


Age distribution by Country:
         country  age
0     Yugoslavia   20
1     Yugoslavia   22
2     Yugoslavia   25
3     Yugoslavia   29
4     Yugoslavia   31
5     Yugoslavia   35
6     Yugoslavia   36
7     Yugoslavia   40
8     Yugoslavia   41
9     Yugoslavia   43
10    Yugoslavia   45
11    Yugoslavia   56
12    Yugoslavia   66
13       Vietnam   19
14       Vietnam   20
15       Vietnam   21
16       Vietnam   22
17       Vietnam   23
18       Vietnam   24
19       Vietnam   25
20       Vietnam   26
21       Vietnam   27
22       Vietnam   29
23       Vietnam   30
24       Vietnam   31
25       Vietnam   33
26       Vietnam   35
27       Vietnam   36
28       Vietnam   37
29       Vietnam   38
...          ...  ...
1251           ?   46
1252           ?   47
1253           ?   48
1254           ?   49
1255           ?   50
1256           ?   51
1257           ?   52
1258           ?   53
1259           ?   54
1260           ?   55
1261           ?   56
1262           ?   57
126

In [43]:
#Question 7: Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

#Subtract Captial Loss from Captial Gain to get Net Capital Gain
pysqldf("SELECT capital_gain as 'Capital Gain', capital_loss as 'Capital Loss', capital_gain - capital_loss as 'Net Capital Gain' FROM sqladb")

,Capital Gain,Capital Loss,Net Capital Gain
0,2174,0,2174
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,14084,0,14084
9,5178,0,5178
